In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv("./data/features3_train.csv")
X = data[data.columns[:-1]]
y = data['VV']

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20)

In [4]:
from Python.sklearn.data_split import *
X_train_bal, y_train_bal = balance_sample(X_train, y_train)
X_train_bal.shape


(40868, 27)

In [6]:
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

clf1 = LogisticRegression(penalty='l2', C=0.001, random_state=1)
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=0)
clf3 = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')
clf4 = SVC(degree=2, probability=True)
clf5 = RandomForestClassifier()

clf_labels = ['LR', 'DT', 'KNN', 'SVC', 'RFC']
clfs = [clf1, clf2, clf3, clf4, clf5]
print('10-fold cross validation:\n')
for clf, label in zip(clfs, clf_labels):
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train, cv=10, scoring='roc_auc')
    print("ROC AUC: %.2f(+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

10-fold cross validation:

ROC AUC: 0.49(+/- 0.04) [LR]
ROC AUC: 0.61(+/- 0.03) [DT]
ROC AUC: 0.55(+/- 0.01) [KNN]
ROC AUC: 0.64(+/- 0.06) [SVC]
ROC AUC: 0.74(+/- 0.03) [RFC]


In [33]:
import xgboost
xg_reg = xgboost.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3,
                              learning_rate = 0.1, max_depth = 5, alpha = 10, n_estimators = 10)
dtrain = xgboost.DMatrix(np.asmatrix(X_train_bal), label=y_train_bal)
dtest = xgboost.DMatrix(np.asmatrix(X_valid), label=y_valid)
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgboost.cv(dtrain=dtrain, params=params, nfold=3,
                    num_boost_round=50,early_stopping_rounds=10,metrics="auc", as_pandas=True, seed=123)
cv_results.head()

[14:11:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:11:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.
[14:11:03] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.683907,0.008143,0.681182,0.006494
1,0.769357,0.018179,0.764218,0.020571
2,0.788939,0.017206,0.783958,0.020102
3,0.813171,0.024255,0.808917,0.026149
4,0.814625,0.022888,0.812025,0.025357


In [35]:
xg_reg = xgboost.train(params=params, dtrain=dtrain, num_boost_round=10)
y_valid_pred = xg_reg.predict(dtest)
y_valid_pred = [0 if x < 0.5 else 1 for x in y_valid_pred]
f1_score(y_valid, y_valid_pred)

[14:11:12] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/objective/regression_obj.cu:170: reg:linear is now deprecated in favor of reg:squarederror.


0.09259259259259259